# Step -week4-

## Wikipediaのグラフを使ってなにか面白いことをしてみよう

今回は、

- 孤立しているページ
- 2つ以上で輪が完成している（それ以外のページからはアクセスできない）ページたち

を探してみる。

### 1. 各ファイルの読み込み、辞書の作成

ファイルサイズが大きく、辞書を作るのに時間がかかってしまうため、元の`links.txt`から以下の二つのファイルを作って、辞書の作成を行いました。

`links_in.txt`：　(ID)\t(IDのページへ繋がっているIDのリスト(,区切り))

`links_out.txt`：　(ID)\t(IDのページから繋がっているIDのリスト(,区切り))

In [1]:
id_to_title = {}
in_links= {} #自分のページ => 自分のページへと繋がっているページのIDのリスト
out_links = {} #自分のページ => 自分のページから繋がっているページのIDのリスト

with open("data/pages.txt") as f:
    for data in f.read().splitlines():
        page = data.split("\t")
        id_to_title[int(page[0])] = page[1]

with open("data/links_in.txt") as f:
    for data in f.read().splitlines():
        link = data.split("\t")
        in_links[int(link[0])] = [int(i) for i in link[1].split(",")]

with open("data/links_out.txt") as f:
    for data in f.read().splitlines():
        link = data.split("\t")
        out_links[int(link[0])] = [int(i) for i in link[1].split(",")]

### 2. 繋がっているページ数を数える

In [2]:
num_of_in = [0] * (len(id_to_title)) #自分のページへと繋がっているページ数
num_of_out = [0] * (len(id_to_title)) #自分のページから繋がっているページ数

for Id, lst in in_links.items():
    num_of_in[Id] = len(lst)
    
for Id, lst in out_links.items():
    num_of_out[Id] = len(lst)
    
# (ID, 入ってくるページ数, 出ていくページ数)のリストを作成
count_pages = list(zip(list(range(len(id_to_title))), num_of_in, num_of_out))

### 3. カウントしたリストを使って、色々調べてみる 

In [3]:
isolated = []

for (Id, in_, out_) in count_pages: #inもoutも0個だったら孤立
    if in_ == 0 and out_ == 0:
        isolated.append(Id)
        
print("【どこともつながっていないページ】（ページ数：%d）\n" %len(isolated))
for Id in isolated[:10]:
    print("ID: ", Id, "\t", id_to_title[Id])
print("...")

【どこともつながっていないページ】（ページ数：1438）

ID:  3965 	 テニス選手の一覧
ID:  10655 	 成人後見
ID:  17000 	 インダストリア
ID:  17002 	 ハイハーバー
ID:  19818 	 9600形
ID:  19824 	 9600型
ID:  21105 	 京急800形
ID:  26834 	 円周率近似値の日
ID:  27197 	 京急600形
ID:  27200 	 京急700形
...


In [4]:
#効率よく調べられるように、まずinとoutが１個のものを集めたリストを作成
tmplist = set()
for (Id, in_, out_) in count_pages: 
    if in_ == 1 and out_ == 1:
        tmplist.add(Id)
        
print("【自分から自分へのみ繋がってるページ】\n")
for Id in tmplist:
    if Id == in_links[Id][0]:
        print("ID: ", Id, "\t", id_to_title[Id])
        

print("\n")
print("【2つでリンクを渡し合っている、かつ、その2つが他とは繋がっていないページ】\n")
for Id in tmplist:
    if int(in_links[Id][0]) in tmplist and int(out_links[Id][0]) in tmplist: #自分と繋がっているページも、tmplistに入っているか
        if in_links[Id] == out_links[Id]: #入ってくるページも出ていくページも同じか
            if Id != in_links[Id][0]: #繋がっている先が自分とは違う
                print(id_to_title[Id], " <=> ", id_to_title[in_links[Id][0]])

【自分から自分へのみ繋がってるページ】

ID:  1481595 	 エトヴォス


【2つでリンクを渡し合っている、かつ、その2つが他とは繋がっていないページ】

日本鍍金材料共同組合  <=>  鍍金の世界
鍍金の世界  <=>  日本鍍金材料共同組合


(2つ以上の孤立して輪になっているページたちを見つけたかったのですが、下のように調べたところそもそもループになっているページたちがこれらしかありませんでした…。)

In [5]:
#リンク先を見ていって、最終的に自分に帰ってきたらTrue、そうでなければFalse
def isin_tmplist(start, Id):
    if start == Id:
        return True
    
    if Id in tmplist:
        if isin_tmplist(start, out_links[Id][0]):
            return True
    return False
        
for Id in tmplist:
    if isin_tmplist(Id, out_links[Id][0]):
        print("ID: ",Id, "\t", id_to_title[Id])

ID:  1451205 	 日本鍍金材料共同組合
ID:  1451208 	 鍍金の世界
ID:  1481595 	 エトヴォス
